<a href="https://colab.research.google.com/github/francoriboratig/TP2-Datos/blob/master/Red%20neuronal%20con%20text%20embeddings%20y%20features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import numpy as np
import tensorflow as tf
import pandas as pd
from google.colab import files
!pip install talos
import talos
import datetime as dt
from sklearn.metrics import roc_auc_score

!pip install -q tensorflow-hub
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")


Version:  2.3.0
Eager mode:  True
Hub version:  0.8.0
GPU is NOT AVAILABLE


In [42]:
# CARGAMOS LOS CSV
url_train_con_features = 'https://raw.githubusercontent.com/francoriboratig/TP2-Datos/master/train-features.csv'
url_test_con_features = 'https://raw.githubusercontent.com/francoriboratig/TP2-Datos/master/test-features.csv'

train_df = pd.read_csv(url_train_con_features)
test_df = pd.read_csv(url_test_con_features)

train_df.head()

,Unnamed: 0,id,keyword,location,text,target,character_count,word_count,contains_word_fire,contains_word_storm,contains_word_flood,contains_word_death,contains_word_love,has_mentions,amount_of_exclamation_marks,amount_of_interrogation_marks,has_link,stopwords_count,contains_countries,contains_cities
0,0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69,13,0,0,0,0,0,False,0,0,False,7,False,False
1,1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38,7,1,0,0,0,0,False,0,0,False,3,True,False
2,2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133,22,0,0,0,0,0,False,0,0,False,13,False,False
3,3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,8,1,0,0,0,0,False,0,0,False,2,False,False
4,4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88,16,1,0,0,0,0,False,0,0,False,10,False,False


In [43]:
# LIMPIAMOS Y CHUSMEAMOS
train_df = train_df.drop(['Unnamed: 0', 'keyword', 'location'], axis = 1)
train_df.shape

(7613, 17)

In [44]:
target_df = train_df.pop('target')

In [45]:
# HACEMOS UN EMBEDDING PROVISTO POR TensorFlow Hub, por fuera del modelo, de esa forma puedo agregarle features al array

embedding_url = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

hub_layer = hub.KerasLayer(embedding_url,
                           input_shape = [], 
                           dtype = tf.string,
                           trainable = True)

train_text_array = np.asarray(train_df['text'])
train_text_embedded = hub_layer(train_text_array)
train_text_embedded_array = train_text_embedded.numpy()
train_text_embedded_df = pd.DataFrame(data = train_text_embedded_array)

train_text_embedded_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.007239,0.156896,1.295307,-0.040780,-0.160369,-1.200443,-0.404513,-0.523973,-0.235587,-0.350222,-0.289882,-0.471979,-0.076219,0.170489,-0.832377,0.022601,0.161255,-0.640868,-0.569074,0.040001
1,-0.498373,1.282888,-0.188361,-0.947347,0.880581,-0.178855,-0.015432,0.383270,-1.508848,1.753281,-0.754933,0.601241,-0.143784,-0.891944,0.432137,1.346741,-1.485925,1.576216,-0.378781,0.942368
2,0.088144,-0.311402,0.604415,1.713081,1.320312,-3.201303,0.844513,0.962179,-1.535314,2.206953,-0.279025,0.010960,-2.228273,0.204312,-2.175010,1.344161,0.118840,-1.238801,-0.255011,0.695272
3,-1.083202,0.694350,-0.293344,0.301429,1.887291,-2.864778,-0.018864,0.176138,-2.388204,1.020881,0.144147,0.802330,-1.326284,-0.055402,-0.268148,1.400147,-1.432997,-1.546905,0.456249,0.117063
4,0.296060,-0.401119,0.260654,-0.133400,0.043096,-0.781329,-0.456910,0.952776,0.359138,0.342683,-1.734295,1.611835,-0.623508,-0.758112,-0.305311,1.009357,-0.759674,-0.749319,-1.216560,0.413135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,0.851313,1.155238,-0.563759,0.338048,1.038724,-0.160978,-0.569604,0.691393,-1.027794,-0.184893,-0.310249,1.777828,-1.333608,-1.097076,-0.556300,1.235541,-0.814311,-0.244320,-0.842489,0.383638
7609,0.586044,1.064535,1.108355,-1.080643,0.978821,-1.538974,-0.011408,0.165366,-0.385796,-0.280098,0.166554,-0.024055,-0.658078,-0.365682,-0.477021,1.355311,-0.433643,0.137504,-1.611735,-0.096220
7610,-1.030287,0.200154,-0.691702,-0.506228,0.250602,0.100836,-0.644162,-1.211905,-1.112232,0.593809,-0.752241,0.449338,0.918436,-0.354921,1.325370,-0.197728,-1.613481,0.592320,1.134160,0.184020
7611,0.192334,-0.517235,-0.762114,-0.045759,2.537380,-0.443210,-0.915873,1.407808,0.293244,-0.688244,-2.480736,1.473341,0.463894,-0.913488,-1.088880,2.593801,-0.944662,-0.532709,-1.173615,0.009434


In [46]:
# MANDALE MECHA CON LOS FEATURES MUÑECO
train_df = train_df.drop(['id', 'text'],axis = 1)
train_df = pd.concat([train_text_embedded_df, train_df], axis=1)

train_df['has_mentions'] = train_df['has_mentions'].astype(int)
train_df['has_link'] = train_df['has_link'].astype(int)
train_df['contains_countries'] = train_df['contains_countries'].astype(int)
train_df['contains_cities'] = train_df['contains_cities'].astype(int)

train_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,character_count,word_count,contains_word_fire,contains_word_storm,contains_word_flood,contains_word_death,contains_word_love,has_mentions,amount_of_exclamation_marks,amount_of_interrogation_marks,has_link,stopwords_count,contains_countries,contains_cities
0,0.007239,0.156896,1.295307,-0.040780,-0.160369,-1.200443,-0.404513,-0.523973,-0.235587,-0.350222,-0.289882,-0.471979,-0.076219,0.170489,-0.832377,0.022601,0.161255,-0.640868,-0.569074,0.040001,69,13,0,0,0,0,0,0,0,0,0,7,0,0
1,-0.498373,1.282888,-0.188361,-0.947347,0.880581,-0.178855,-0.015432,0.383270,-1.508848,1.753281,-0.754933,0.601241,-0.143784,-0.891944,0.432137,1.346741,-1.485925,1.576216,-0.378781,0.942368,38,7,1,0,0,0,0,0,0,0,0,3,1,0
2,0.088144,-0.311402,0.604415,1.713081,1.320312,-3.201303,0.844513,0.962179,-1.535314,2.206953,-0.279025,0.010960,-2.228273,0.204312,-2.175010,1.344161,0.118840,-1.238801,-0.255011,0.695272,133,22,0,0,0,0,0,0,0,0,0,13,0,0
3,-1.083202,0.694350,-0.293344,0.301429,1.887291,-2.864778,-0.018864,0.176138,-2.388204,1.020881,0.144147,0.802330,-1.326284,-0.055402,-0.268148,1.400147,-1.432997,-1.546905,0.456249,0.117063,65,8,1,0,0,0,0,0,0,0,0,2,0,0
4,0.296060,-0.401119,0.260654,-0.133400,0.043096,-0.781329,-0.456910,0.952776,0.359138,0.342683,-1.734295,1.611835,-0.623508,-0.758112,-0.305311,1.009357,-0.759674,-0.749319,-1.216560,0.413135,88,16,1,0,0,0,0,0,0,0,0,10,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,0.851313,1.155238,-0.563759,0.338048,1.038724,-0.160978,-0.569604,0.691393,-1.027794,-0.184893,-0.310249,1.777828,-1.333608,-1.097076,-0.556300,1.235541,-0.814311,-0.244320,-0.842489,0.383638,83,11,0,0,0,0,0,0,0,0,1,6,0,0
7609,0.586044,1.064535,1.108355,-1.080643,0.978821,-1.538974,-0.011408,0.165366,-0.385796,-0.280098,0.166554,-0.024055,-0.658078,-0.365682,-0.477021,1.355311,-0.433643,0.137504,-1.611735,-0.096220,125,20,1,0,0,0,0,1,0,0,0,13,0,0
7610,-1.030287,0.200154,-0.691702,-0.506228,0.250602,0.100836,-0.644162,-1.211905,-1.112232,0.593809,-0.752241,0.449338,0.918436,-0.354921,1.325370,-0.197728,-1.613481,0.592320,1.134160,0.184020,65,8,0,0,0,0,0,0,0,1,1,4,0,0
7611,0.192334,-0.517235,-0.762114,-0.045759,2.537380,-0.443210,-0.915873,1.407808,0.293244,-0.688244,-2.480736,1.473341,0.463894,-0.913488,-1.088880,2.593801,-0.944662,-0.532709,-1.173615,0.009434,137,19,0,0,0,0,0,0,0,0,0,12,1,1


In [47]:
def modelo(x_train, y_train, x_test, y_test, params):
  
  cantidad_columnas = x_train.shape[1]
  input_shape = tuple([cantidad_columnas])

  model = tf.keras.Sequential()
  model.add(tf.keras.layers.LayerNormalization(input_shape = input_shape))
  model.add(tf.keras.layers.Dense(params['first neuron units'], activation = params['neuron activation']))
  model.add(tf.keras.layers.Dropout(rate=0.2))
  model.add(tf.keras.layers.LayerNormalization())
  model.add(tf.keras.layers.Dense(params['second neuron units'], activation = params['neuron activation']))
  model.add(tf.keras.layers.Dense(1))

  
  model.compile(optimizer = 'adam',
              loss = tf.keras.losses.BinaryCrossentropy(from_logits = True),
              metrics = ['accuracy'])
  
  out = model.fit(x_train, y_train, batch_size=params['batch size'],
                    epochs = params['epochs'],
                    verbose = 1,validation_data = (x_test,y_test))

  return out, model


p = {
    'first neuron units': [1,2,4,8,20,32,64,128],
    'second neuron units': [1,2,4,8,20,32,64,128],
    'neuron activation': ['relu', 'elu', 'softmax', 'tanh', 'exponential'],
    'epochs': [1, 2, 4, 5, 10],
    'batch size': [5, 10, 50, 100]
}

In [ ]:
t = talos.Scan(train_df.to_numpy(),
               target_df.to_numpy(),
               params = p,
               model = modelo,
               experiment_name = 'diabetes',
               time_limit = (dt.datetime.now() + dt.timedelta(minutes = 1500)).strftime("%Y-%m-%d %H:%M"),
               random_method = 'quantum')







  0%|          | 0/6400 [00:00<?, ?it/s]

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1058/1066 [============================>.] - ETA: 0s - loss: 0.6878 - accuracy: 0.5679WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7fae1f17a2f0> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1066/1066 [==============================] - 2s 2ms/step - loss: 0.6693 - accuracy: 0.5703 - val_loss: 0.6368 - val_accuracy: 0.5959






  2%|▏         | 127/6400 [05:46<4:47:16,  2.75s/it]

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [ ]:
experiment_data = t.data
experiment_data()

In [ ]:
maxim=experiment_data['val_accuracy'].idxmax()
experiment_data.iloc[maxim]

In [ ]:
experiment_data.to_csv('experiment.csv')
files.download('experiment.csv')


In [ ]:
# SE DIVIDE UNA PARTE DEL SET PARA PODER EVALUAR LOS RESULTADOS
train_x, validation_x, train_y, validation_y = train_test_split(train_df, target_df, test_size=0.3, random_state=0)

In [ ]:
# ARMAMOS EL MODELO
cantidad_columnas = train_x.shape[1]
input_shape = tuple([cantidad_columnas])

model = tf.keras.Sequential()
model.add(tf.keras.layers.LayerNormalization(input_shape = input_shape))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dropout(rate=0.2))
model.add(tf.keras.layers.LayerNormalization())
model.add(tf.keras.layers.Dense(14, activation = 'relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

In [ ]:
# COMPILAMOS Y ENTRENAMOS

epochs = 300

model.save_weights('model.h5') #esto es para luego resetear el entrenamiento

model.compile(optimizer = 'adam',
              loss = tf.keras.losses.BinaryCrossentropy(from_logits = True),
              metrics = ['accuracy'])

history = model.fit(train_x, train_y, batch_size=1024,
                    epochs = epochs,
                    verbose = 1)

In [ ]:
# EVALUAMOS CON EL SET DE VALIDACION
loss, precision = model.evaluate(validation_x, validation_y)
print('Precisión: %.2f' % (precision*100))

In [ ]:
# SE REENTRENA CON EL SET COMPLETO

model.load_weights('model.h5') #se resetea el entrenamiento previo

history = model.fit(train_df, target_df, batch_size = 1024,
                   epochs = epochs,
                   verbose = 1)

In [ ]:
# ARMAMOS LA PREDICCIÓN
test_text_array = np.asarray(test_df['text'])
test_text_embedded = hub_layer(test_text_array)
test_text_embedded_array = test_text_embedded.numpy()
test_text_embedded_df = pd.DataFrame(data = test_text_embedded_array)

test_df = pd.concat([test_text_embedded_df, test_df], axis=1)

test_df = test_df.drop(['Unnamed: 0', 'keyword', 'location', 'text'],axis = 1)

test_df['has_mentions'] = test_df['has_mentions'].astype(int)
test_df['has_link'] = test_df['has_link'].astype(int)
test_df['contains_countries'] = test_df['contains_countries'].astype(int)
test_df['contains_cities'] = test_df['contains_cities'].astype(int)

# ARMAMOS LOS DATOS EN EL FORMATO QUE PIDE KAGGLE
export_df = pd.DataFrame()
export_df['id'] = test_df.pop('id')
export_df = export_df.set_index('id')
export_df['target'] = (model.predict(test_df) > 0.5).astype("int32")

export_df

In [ ]:
# DOWNLOAD !
export_df.to_csv('submit.csv')
#files.download('submit.csv')